In [1]:
import random
import math
import pandas as pd
import numpy as np
from keras.models import model_from_json
from numpy import array

Using TensorFlow backend.


In [2]:
class boundaries:
    max_D = {}
    min_D = {}
    max_N = {}
    min_N = {}

    def __init__(self, max_D, min_D, max_N, min_N): 
        self.max_D = max_D 
        self.min_D = min_D
        self.max_N = max_N
        self.min_N = min_N

def read_csv():
    #Leyendo archivo
    df = pd.read_csv('Interpolation/InterpolatedDenMonth.csv')
    df_N = pd.read_csv('Interpolation/InterpolatedNumMonth.csv')
    global ufcf, dfcf, max_D, max_N, min_D, min_N
    global x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12
    max_D = {'D REVENUE':df['D REVENUE'].max(), 'U CR':df['U CR'].max(), 'D OE':df['D OE'].max(), 
       'D NOI':df['D NOI'].max(),'U CAPEX':df['U CAPEX'].max(), 'U CWK':df['U CWK'].max()} 
    min_D = {'D REVENUE':df['D REVENUE'].min(), 'U CR':df['U CR'].min(), 'D OE':df['D OE'].min(), 
       'D NOI':df['D NOI'].min(),'U CAPEX':df['U CAPEX'].min(), 'U CWK':df['U CWK'].min()}
    max_N = {'U REVENUE':df_N['U REVENUE'].max(), 'D CR':df_N['D CR'].max(), 'U OE':df_N['U OE'].max(), 
       'U NOI':df_N['U NOI'].max(),'D CAPEX':df_N['D CAPEX'].max(), 'D CWK':df_N['D CWK'].max()} 
    min_N = {'U REVENUE':df_N['U REVENUE'].min(), 'D CR':df_N['D CR'].min(), 'U OE':df_N['U OE'].min(), 
       'U NOI':df_N['U NOI'].min(),'D CAPEX':df_N['D CAPEX'].min(), 'D CWK':df_N['D CWK'].min()}
   
    filas_d, columnas_d = df.count()-20, len(df.columns)-1
    dataset_D = df.values
    #Variables a pasar a la funcion generate_population
    dfcf = float(dataset_D[filas_d, columnas_d][1])
    filas_n, columnas_n = df_N.count()-20, len(df_N.columns)-1
    dataset_N = df_N.values
    ufcf = float(dataset_N[filas_d, columnas_d][1])
    
    boundaries_x = boundaries(max_D, min_D, max_N, min_N)
        
    
    ##Calculo de las x
    x1 = df_N['U REVENUE'].corr(df_N['U FCF'])
    x2 = df_N['D CR'].corr(df_N['U FCF'])
    x3 = df_N['U OE'].corr(df_N['U FCF'])
    x4 = df_N['U NOI'].corr(df_N['U FCF'])
    x5 = df_N['D CAPEX'].corr(df_N['U FCF'])
    x6 = df_N['D CWK'].corr(df_N['U FCF'])
    x7 = df['D REVENUE'].corr(df['D FCF'])
    x8 = df['U CR'].corr(df['D FCF'])
    x9 = df['D OE'].corr(df['D FCF'])
    x10 = df['D NOI'].corr(df['D FCF'])
    x11 = df['U CAPEX'].corr(df['D FCF'])
    x12 = df['U CWK'].corr(df['D FCF'])

    return boundaries_x


def generate_population(boundarie, size):
    population = []
    min_N = boundarie.min_N
    max_N = boundarie.max_N
    min_D = boundarie.min_D
    max_D = boundarie.max_D
    for i in range(size):
        individual = {
            "w1": random.uniform(min_N['U REVENUE'], max_N['U REVENUE']),
            "w2": random.uniform(min_N['D CR'], max_N['D CR']),
            "w3": random.uniform(min_N['U OE'], max_N['U OE']),
            "w4": random.uniform(min_N['U NOI'], max_N['U NOI']),
            "w5": random.uniform(min_N['D CAPEX'], max_N['D CAPEX']),
            "w6": random.uniform(min_N['D CWK'], max_N['D CWK']),
            "w7": random.uniform(min_D['D REVENUE'], max_D['D REVENUE']),
            "w8": random.uniform(min_D['U CR'], max_D['U CR']),
            "w9": random.uniform(min_D['D OE'], max_D['D OE']),
            "w10": random.uniform(min_D['D NOI'], max_D['D NOI']),
            "w11": random.uniform(min_D['U CAPEX'], max_D['U CAPEX']),
            "w12": random.uniform(min_D['U CWK'], max_D['U CWK']),
        }    
        population.append(individual)
    return population

def apply_function(individual):
    # Numerador
    # load json and create model
    json_file = open('modelnum.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model_num = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model_num.load_weights("modelnum.h5")
    # Denominador
    json_file = open('modelden.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model_den = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model_den.load_weights("modelden.h5")
    w1 = individual["w1"]
    w2 = individual["w2"]
    w3 = individual["w3"]
    w4 = individual["w4"]
    w5 = individual["w5"]
    w6 = individual["w6"]
    w7 = individual["w7"]
    w8 = individual["w8"]
    w9 = individual["w9"]
    w10 = individual["w10"]
    w11 = individual["w11"]
    w12 = individual["w12"]
    w1_std = (w1-min_N['U REVENUE'])/(max_N['U REVENUE']-min_N['U REVENUE'])
    w2_std = (w2-min_N['D CR'])/(max_N['D CR']-min_N['D CR'])
    w3_std = (w3-min_N['U OE'])/(max_N['U OE']-min_N['U OE'])
    w4_std = (w4-min_N['U NOI'])/(max_N['U NOI']-min_N['U NOI'])
    w5_std = (w5-min_N['D CAPEX'])/(max_N['D CAPEX']-min_N['D CAPEX'])
    w6_std = (w6-min_N['D CWK'])/(max_N['D CWK']-min_N['D CWK'])
    w7_std = (w7-min_D['D REVENUE'])/(max_D['D REVENUE']-min_D['D REVENUE'])
    w8_std = (w8-min_D['U CR'])/(max_D['U CR']-min_D['U CR'])
    w9_std = (w9-min_D['D OE'])/(max_D['D OE']-min_D['D OE'])
    w10_std = (w10-min_D['D NOI'])/(max_D['D NOI']-min_D['D NOI'])
    w11_std = (w11-min_D['U CAPEX'])/(max_D['U CAPEX']-min_D['U CAPEX'])
    w12_std = (w12-min_D['U CWK'])/(max_D['U CWK']-min_D['U CWK'])
    w_numerador = [[w1_std, w2_std, w3_std, w4_std, w5_std, w6_std]]
    w_denominador = [[w7_std, w8_std, w9_std, w10_std, w11_std, w12_std]]
    Xnewnum = array(w_numerador)
    Xnewden = array(w_denominador)
    # make a prediction
    numerador = loaded_model_num.predict(Xnewnum) 
    denominador = loaded_model_den.predict(Xnewden)
    function_costo1 = ufcf/dfcf
    function_costo2 = numerador/denominador
    function_costo2num = function_costo2[0][0]
    function_result = (function_costo2num - function_costo1)/function_costo1
    return function_result

def choice_by_roulette(sorted_population, fitness_sum):
    offset = 0
    normalized_fitness_sum = fitness_sum

    lowest_fitness = apply_function(sorted_population[0])
    if lowest_fitness < 0:
        offset = -lowest_fitness
        normalized_fitness_sum += offset * len(sorted_population)

    draw = random.uniform(0, 1)

    accumulated = 0
    for individual in sorted_population:
        fitness = apply_function(individual) + offset
        probability = fitness / normalized_fitness_sum
        accumulated += probability

        if draw <= accumulated:
            return individual

def sort_population_by_fitness(population):
    return sorted(population, key=apply_function)


def crossover(individual_a, individual_b):
    w1a = individual_a["w1"]
    w2a = individual_a["w2"]
    w3a = individual_a["w3"]
    w4a = individual_a["w4"]
    w5a = individual_a["w5"]
    w6a = individual_a["w6"]
    w7a = individual_a["w7"]
    w8a = individual_a["w8"]
    w9a = individual_a["w9"]
    w10a = individual_a["w10"]
    w11a = individual_a["w11"]
    w12a = individual_a["w12"]
    

    w1b = individual_b["w1"]
    w2b = individual_b["w2"]
    w3b = individual_b["w3"]
    w4b = individual_b["w4"]
    w5b = individual_b["w5"]
    w6b = individual_b["w6"]
    w7b = individual_b["w7"]
    w8b = individual_b["w8"]
    w9b = individual_b["w9"]
    w10b = individual_b["w10"]
    w11b = individual_b["w11"]
    w12b = individual_b["w12"]

    result_crossover = {"w1": (w1a + w1b) / 2, "w2": (w2a + w2b) / 2, "w3": (w3a + w3b) / 2, "w4": (w4a + w4b) / 2, 
    "w5": (w5a + w5b) / 2, "w6": (w6a + w6b) / 2, "w7": (w7a + w7b) / 2, "w8": (w8a + w8b) / 2,
    "w9": (w9a + w9b) / 2, "w10": (w10a + w10b) / 2, "w11": (w11a + w11b) / 2, "w12": (w12a + w12b) / 2}
    return result_crossover


def mutate(individual):
    min_value = -1
    max_value = 1
    next_w1 = individual["w1"] * (1+random.uniform(min_value, max_value)) 
    next_w2 = individual["w2"] * (1+random.uniform(min_value, max_value))
    next_w3 = individual["w3"] * (1+random.uniform(min_value, max_value))
    next_w4 = individual["w4"] * (1+random.uniform(min_value, max_value))
    next_w5 = individual["w5"] * (1+random.uniform(min_value, max_value))
    next_w6 = individual["w6"] * (1+random.uniform(min_value, max_value))
    next_w7 = individual["w7"] * (1+random.uniform(min_value, max_value))
    next_w8 = individual["w8"] * (1+random.uniform(min_value, max_value))
    next_w9 = individual["w9"] * (1+random.uniform(min_value, max_value))
    next_w10 = individual["w10"] * (1+random.uniform(min_value, max_value))
    next_w11 = individual["w11"] * (1+random.uniform(min_value, max_value))
    next_w12 = individual["w12"] * (1+random.uniform(min_value, max_value))

    result_mutation = {"w1": next_w1, "w2": next_w2, "w3": next_w3, "w4": next_w4, "w5": next_w5,
    "w6": next_w6, "w7": next_w7, "w8": next_w8, "w9": next_w9, "w10": next_w10, "w11": next_w11, "w12": next_w12}
    return result_mutation


def make_next_generation(previous_population):
    next_generation = []
    sorted_by_fitness_population = sort_population_by_fitness(previous_population)
    population_size = len(previous_population)
    fitness_sum = sum(apply_function(individual) for individual in previous_population)

    for i in range(population_size):
        first_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)
        second_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)

        individual = crossover(first_choice, second_choice)
        individual = mutate(individual)
        next_generation.append(individual)

    return next_generation

In [3]:
def main():
    generations = 10
    boundaries_x = read_csv()    
    population = generate_population(boundarie=boundaries_x, size=10)

    i = 1
    while True:
        print(f"🧬 GENERATION {i}")

        for individual in population:
             print(individual, apply_function(individual))

        if i == generations:
            break

        i += 1

        population = make_next_generation(population)

    best_individual = sort_population_by_fitness(population)[-1]
    print("\n🔬 FINAL RESULT")
    print(best_individual, (apply_function(best_individual)*100), "%")

In [4]:
if __name__ == "__main__":
    main()

🧬 GENERATION 1
{'w1': 2870.250233312448, 'w2': 1216.9237290038438, 'w3': 620.9121037842148, 'w4': 412.40552111397733, 'w5': 253.60851135484586, 'w6': 5296.875605815545, 'w7': 1821.1838576149457, 'w8': 1017.4933652978921, 'w9': 658.1995316362998, 'w10': 311.50108394217307, 'w11': 358.6418801631906, 'w12': 1987.7764457791884} -0.31292477377084976
{'w1': 2138.4364126144765, 'w2': 1426.1133573596449, 'w3': 451.24284467084016, 'w4': 212.68216612887483, 'w5': 660.9610674382111, 'w6': 3161.6385820498544, 'w7': 2588.650022961754, 'w8': 1092.5218158467842, 'w9': 434.73371895841683, 'w10': 624.7153284914176, 'w11': 543.9350046203382, 'w12': 2775.038542730266} -0.07833417003456807
{'w1': 2360.9450819514686, 'w2': 674.2854316501976, 'w3': 627.2242640198731, 'w4': 433.1376780217822, 'w5': 298.6006255919954, 'w6': 2317.5171872326023, 'w7': 1863.8771903209333, 'w8': 696.884741838295, 'w9': 525.2143923051391, 'w10': 206.34393773904208, 'w11': 242.12270251421137, 'w12': 4151.235874230105} -0.1829470598

{'w1': 4846.597899438292, 'w2': 2043.3271203197812, 'w3': 231.45443312579604, 'w4': 8.943593656430176, 'w5': 831.9459421234291, 'w6': 188.15927324698544, 'w7': 7398.2500312051225, 'w8': 2865.7069600387113, 'w9': 631.7896097687736, 'w10': 284.98732258957205, 'w11': 384.89182077503733, 'w12': 130.84720746634446} 1.0474154887323637
{'w1': 121.51979405100711, 'w2': 2001.4558577727983, 'w3': 22.72854546973304, 'w4': 205.9972193059645, 'w5': 349.4693010711321, 'w6': 3590.9225151778287, 'w7': 2145.751527891555, 'w8': 930.6454559867154, 'w9': 789.3762135822411, 'w10': 19.464751653499235, 'w11': 176.1109610737517, 'w12': 3099.5985742663806} -1.000789747521474
{'w1': 5329.09161327857, 'w2': 2003.737137449694, 'w3': 208.3879330127346, 'w4': 289.37844937972415, 'w5': 161.4504606556123, 'w6': 406.39073824734515, 'w7': 3808.3397666968967, 'w8': 970.7449204119619, 'w9': 807.2034783589577, 'w10': 360.53145787024215, 'w11': 320.40711685485525, 'w12': 4301.105266237817} 0.41734088024687654
{'w1': 255.46

🧬 GENERATION 6
{'w1': 7884.5147388335545, 'w2': 324.3467038039956, 'w3': 674.9457226978402, 'w4': 124.26283579996554, 'w5': 749.9997105065967, 'w6': 1230.3100688515096, 'w7': 1273.7720226665997, 'w8': 853.8089959619106, 'w9': 467.9465486441859, 'w10': 857.9722706849376, 'w11': 339.30395344684683, 'w12': 2195.048673573209} 0.39053322740662366
{'w1': 14017.6093368288, 'w2': 355.40496763526744, 'w3': 477.947383981744, 'w4': 127.45175070843756, 'w5': 576.7966240455538, 'w6': 3162.407856114624, 'w7': 2872.518281424883, 'w8': 2598.962964755741, 'w9': 913.6728740461992, 'w10': 334.66589099691817, 'w11': 608.2402254674093, 'w12': 154.36189512626115} 0.9219602877016084
{'w1': 1415.6043666676233, 'w2': 414.5226137023935, 'w3': 85.03366395186357, 'w4': 23.392511210775364, 'w5': 263.5291830408941, 'w6': 7060.622718260314, 'w7': 4718.195431569485, 'w8': 5665.81464215749, 'w9': 1377.8642086709951, 'w10': 1.1741221453748767, 'w11': 681.0514022381053, 'w12': 780.5512849410843} -0.41194843793825575
{'w

{'w1': 9464.730958110764, 'w2': 3220.651549018251, 'w3': 242.80136419667463, 'w4': 201.7342976695373, 'w5': 431.7307440804646, 'w6': 2699.918130020975, 'w7': 2883.651215554354, 'w8': 815.5918409793103, 'w9': 153.64328387522588, 'w10': 138.3218706595569, 'w11': 587.1219457225969, 'w12': 1898.3958023772275} 0.5414654569941498
{'w1': 9339.327231561188, 'w2': 242.1565207119189, 'w3': 541.0295562537223, 'w4': 68.6659621373868, 'w5': 506.0121940113751, 'w6': 748.5300369526901, 'w7': 10754.109137570862, 'w8': 1256.45923994156, 'w9': 359.89769937772985, 'w10': 92.17848902330564, 'w11': 282.81352744863347, 'w12': 653.6482695101253} 1.6943175313899386
{'w1': 6069.482299971631, 'w2': 857.5288024067597, 'w3': 33.9601261180724, 'w4': 77.40459736069103, 'w5': 875.0006938807287, 'w6': 35.96343250069494, 'w7': 11950.3815085323, 'w8': 2229.5088388077957, 'w9': 1348.002910382425, 'w10': 146.22711859690438, 'w11': 1314.6039578270409, 'w12': 1078.1901336689868} 0.866633144611244
{'w1': 4410.027068252462, 


🔬 FINAL RESULT
{'w1': 14177.461306200097, 'w2': 72.10272436003976, 'w3': 73.25952898323922, 'w4': 218.17467083840418, 'w5': 475.56655583118186, 'w6': 1108.566547283183, 'w7': 27003.46788487982, 'w8': 7319.505734532383, 'w9': 1721.4951494274083, 'w10': 180.4441185698786, 'w11': 74.306455960519, 'w12': 2223.055499700805} 253.20785802702483 %
